# Embed documents

1. 创建 Embed

In [ ]:
from langchain_openai import OpenAIEmbeddings

e_model = OpenAIEmbeddings()
ebeddings = e_model.embed_documents(
     [
        "你好",
        "你好啊",
        "你叫什么名字?",
        "我叫王大锤",
        "很高兴认识你大锤",
    ]
)
ebeddings

# [[0.0003300555516486924,
#   -0.0060982073022533625,
#   -0.002391764280324817,
#   -0.029246050037373125,
#   -0.04374236023339329,
#   0.013926087440424438,
#   -0.022441402872292373,
#   -0.008470963986649177,
#   -0.015218590925080091,
#   -0.019387547613221756,
#   0.03550582132294638,
#   0.0014572338484260309,
#   0.004808871711447434,
#   -0.002569166801570725,
#   -0.008337912444960724,
#   -0.01678986906851155,
#   0.03411194430377691,
#   -0.01516790462348449,
#   0.0182597736774292,
#   -0.020008454807767872,
#   -4.872502076702215e-07,
#   0.0012521123724103544,
#   0.009281946674824012,
#   0.0008537489641078078,
#   -0.007767690155026299,
# ...

2. Embed query

In [ ]:
embedded_query = e_model.embed_query("这段对话中提到了什么名字?")
embedded_query[:5]

# [0.004047733608954853,
#  0.0009470974041103827,
#  0.029676812733286713,
#  -0.00633586830824844,
#  -0.02480508870743322]

3. 嵌入向量缓存

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import  LocalFileStore
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

u_embeddings = OpenAIEmbeddings()
fs = LocalFileStore("../cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    u_embeddings,
    fs,
    namespace=u_embeddings.model,
)
list(fs.yield_keys())

# ['text-embedding-ada-002c63ea318-3b5d-533b-960b-46434f8b3c22',
#  'text-embedding-ada-002f05b40fb-a095-546e-9c5d-49e069720828',
#  'text-embedding-ada-0024250f053-4b1e-5c34-927d-a7857749217f',
#  'text-embedding-ada-002b0c54c27-a009-50b4-9ccc-661d5478b195',
#  'text-embedding-ada-002e94acbbe-7d17-5331-8310-4e37bdc56d31',
#  'text-embedding-ada-0029286d74c-b3fc-56ff-8b08-9071a193f724']

4. 加载文档，切分文档，将切分文档向量化病存储在缓存中

In [ ]:
raw_documents = TextLoader("../assets/letter.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=600,chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
! pip install faiss-cup

In [ ]:
from langchain.vectorstores import  FAISS
%timeit -r  1 -n 1 db= FAISS.from_documents(documents,cached_embeddings)

#查看缓存中的键
list(fs.yield_keys())

# ['text-embedding-ada-002c63ea318-3b5d-533b-960b-46434f8b3c22',
#  'text-embedding-ada-0024250f053-4b1e-5c34-927d-a7857749217f',
#  'text-embedding-ada-002b0c54c27-a009-50b4-9ccc-661d5478b195',
#  'text-embedding-ada-002e94acbbe-7d17-5331-8310-4e37bdc56d31',
#  'text-embedding-ada-0029286d74c-b3fc-56ff-8b08-9071a193f724']

In [ ]:
到 4.8